In [1]:
!pip install transformers
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
data_path = '/Users/charansaisadla/Desktop/ub classes/SEM2/NLP/data project/'
df=pd.read_csv(data_path+"input.csv")


In [32]:
df['class'] = df['class'].apply(lambda x: 1 if x==7 else 0)
df['text'][7000]

'endoftext16 japan experiences every traveler needs to enjoy their national parks and forests and the cultural diversity that surrounds themkanto region the largest island city in japan tokyo boasts an amazing culture that has influenced the way culture works in culturerich countries throughout the world kita culture and culture culture are closely tied with the culture of nature and the culture culture of the culture and the cultural culture culture in this area is one of japan culture culture cultures culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture culture cul

In [4]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


Here we are using the BERTforsequenceClassification which can be used as classifier,here we are using from_pretained is which there are several models are available for bert ex:bert-large-cased,bert-base-cased with several change in parameters but the basic and efficient one we are using bert_base-uncasedd here.

The number of labels mentioned here is 2 which denotes that it has to classfiy into 2 classes.

The tokenizer we are using BERTTokenizer which can be used as tokenize the input and feed it into the BERT.

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

so here we are defining a function in which is going to tokenize and encode together whihc passing the inputs text,tokenizer,maxlenght.

encode_plus() method of the tokenizer object. The encode_plus() method takes in the text input, adds special tokens to mark the beginning and end of the sequence, truncates or pads the sequence to the specified max_length, and returns a dictionary containing the encoded sequence, the attention mask, and the token type IDs.

so usually needed the encoded inputs and attentionmask to know which we can feed and which we can skip.



In [6]:
import torch

def tokenize_and_encode(text, tokenizer, max_length):
    input_ids = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_length, padding='max_length', truncation=True, return_token_type_ids=False, return_attention_mask=True, return_tensors='pt')
    return input_ids['input_ids'], input_ids['attention_mask']


input_ids = torch.cat(input_ids, dim=0): Concatenates the list of encoded input sequences into a single PyTorch tensor along the 0th dimension.
attention_masks = torch.cat(attention_masks, dim=0): Concatenates the list of attention masks into a single PyTorch tensor along the 0th dimension.
labels = torch.tensor(labels): Converts the list of class labels to a PyTorch tensor.

TensorDataset object using the input sequences, attention masks, and labels that we have converted to PyTorch tensors. This TensorDataset object then serves as the input to the DataLoader.

The DataLoader iterates over the TensorDataset object in batches, shuffling the data if shuffle=True. During each iteration, the DataLoader returns a batch of input sequences, attention masks, and labels, which can then be used to train or evaluate the model.

By using DataLoader to iterate over the dataset, we can effectively process large amounts of data in batches, while minimizing the amount of memory required to store the entire dataset in memory at once.

In [7]:
def get_dataloader(df, tokenizer, max_length, batch_size, shuffle=True):
    texts = df['text'].values
    labels = df['class'].values
    
    input_ids = []
    attention_masks = []
    
    for text in texts:
        input_id, attention_mask = tokenize_and_encode(text, tokenizer, max_length)
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    
    dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    
    return dataloader


In [8]:
def train_model(model, train_dataloader, optimizer, device):
    model.train()
    
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_masks, labels = batch
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        loss.backward()
        optimizer.step()
    
    return model


In [9]:
def evaluate_model(model, val_dataloader, device):
    model.eval()
    val_loss, val_accuracy = 0, 0
    n_val_steps = 0
    
    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_masks, labels = batch

            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            preds = logits.argmax(dim=1)
            val_accuracy += (preds == labels).float().mean().item()
            n_val_steps += 1

    val_loss /= n_val_steps
    val_accuracy /= n_val_steps

    return val_loss, val_accuracy



In [10]:
MAX_LENGTH = 128
BATCH_SIZE = 16
NUM_EPOCHS = 5
LEARNING_RATE = 2e-5

device = torch.device("mps")
model = BertForSequenceClassification.from_pretrained("bert-base-cased").to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

train_dataloader = get_dataloader(train_df, tokenizer, MAX_LENGTH, BATCH_SIZE)
val_dataloader = get_dataloader(test_df, tokenizer, MAX_LENGTH, BATCH_SIZE)

for epoch in range(NUM_EPOCHS):
    train_model(model, train_dataloader, optimizer, device)
    val_loss, val_accuracy = evaluate_model(model, val_dataloader, device)
    print(f"Epoch {epoch + 1} Val Loss: {val_loss:.3f}, Val Accuracy: {val_accuracy:.3f}")


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch 1 Val Loss: 0.176, Val Accuracy: 0.929
Epoch 2 Val Loss: 0.079, Val Accuracy: 0.973
Epoch 3 Val Loss: 0.062, Val Accuracy: 0.986
Epoch 4 Val Loss: 0.058, Val Accuracy: 0.989
Epoch 5 Val Loss: 0.156, Val Accuracy: 0.978


In [1]:
import torch
import math
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())


True
True


In [36]:
# Assume you have a trained model named `model`, a tokenizer named `tokenizer`,
# and a device named `device` that specifies the hardware to use (e.g. "cuda" or "cpu").

input_text = "Cristiano Ronaldo is a Portuguese professional footballer widely regarded as one of the greatest players of all time. He was born on February 5, 1985, in Madeira, Portugal. Ronaldo began his football career at a young age, playing for Andorinha and later moving on to play for Nacional.At the age of just 16, Ronaldo signed with Sporting CP, one of the top football clubs in Portugal. He quickly made a name for himself, scoring several impressive goals and helping his team win important matches. In 2003, at the age of 18, Ronaldo signed with Manchester United, one of the most prestigious football clubs in the world."
encoded_input = tokenizer(input_text, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors='pt')
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()


if predicted_class==1:
    print("Predicted class: Human")
else:
    print("machine")


machine


In [37]:
import torch

# Assuming your model is named 'my_model'
# Save the model's state dictionary
torch.save(model.state_dict(), 'BERT p2.pt')